In [1]:
from pushcart.sources.rest_api.spark import request_dataframe
import pushcart.transformations.split_nested.spark as ps
from pyspark.sql import SparkSession

import pyspark.sql.functions as F


23/08/17 16:49:09 WARN Utils: Your hostname, laptop resolves to a loopback address: 127.0.1.1; using 192.168.2.4 instead (on interface wlan0)
23/08/17 16:49:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/17 16:49:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark = SparkSession.builder.getOrCreate()


In [3]:
data_schema = "array<struct<per_page:int, total:int, total_pages:int, page:int, data:array<struct<last_name:string, avatar:string, id:int, first_name:string, email:string>>>>"

result_df = request_dataframe(
    url="https://reqres.in/api/users",
    params={"page": 2},
    method="GET",
    schema=data_schema,
)
result_df.show(truncate=False)


+---------------------------+-------+----+-----------+----+---------+------+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|url                        |headers|auth|params     |data|json_data|method|options|result                                                                                                                                                                                                                                                                                 

2023-08-17 16:49:15.937 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:202 - URL: https://reqres.in/api/users
2023-08-17 16:49:15.937 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:203 - Parameters: {'page': '2'}
2023-08-17 16:49:15.937 | INFO     | pushcart.sources.rest_api.request:_build_no_auth_client:110 - Using no authentication.
2023-08-17 16:49:15.944 | DEBUG    | pushcart.sources.rest_api.request:_add_data_for_get_request:236 - Added JSON dump of data to GET request body.
2023-08-17 16:49:16.005 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:358 - Returning JSON response:
2023-08-17 16:49:16.005 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:360 - [{'page': 2, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 7, 'email': 'michael.lawson@reqres.in', 'first_name': 'Michael', 'last_name': 'Lawson', 'avatar': 'https://reqres.in/img/faces/7-image.jpg'}, {'id': 8, 'email': 'lindsay.fergu

In [4]:
split_tables = ps.split(result_df, ["result"])
split_tables

2023-08-17 16:49:16.073 | INFO     | pushcart.transformations.split_nested.spark:_split_root:296 - Creating dataframe to serve as base for splitting.
2023-08-17 16:49:16.127 | INFO     | pushcart.transformations.split_nested.spark:_create_split_table:254 - Splitting column result from original dataframe.
2023-08-17 16:49:16.167 | INFO     | pushcart.transformations.split_nested.spark:_split_children:337 - Created split dataframe for result column.


{'root': DataFrame[url: string, headers: map<string,string>, auth: map<string,string>, params: map<string,string>, data: string, json_data: map<string,string>, method: string, options: map<string,string>, result_key: string],
 'result': DataFrame[result_key: string, result_pos: int, result_col: struct<per_page:int,total:int,total_pages:int,page:int,data:array<struct<last_name:string,avatar:string,id:int,first_name:string,email:string>>>]}

In [5]:
split_tables["root"].show(truncate=False)

+---------------------------+-------+----+-----------+----+---------+------+-------+----------------------------------------+
|url                        |headers|auth|params     |data|json_data|method|options|result_key                              |
+---------------------------+-------+----+-----------+----+---------+------+-------+----------------------------------------+
|https://reqres.in/api/users|{}     |{}  |{page -> 2}|    |{}       |GET   |{}     |dd4fb749e98bd6b3db6218a04a9b8e96b349b6d8|
+---------------------------+-------+----+-----------+----+---------+------+-------+----------------------------------------+



2023-08-17 16:49:16.583 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:202 - URL: https://reqres.in/api/users
2023-08-17 16:49:16.583 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:203 - Parameters: {'page': '2'}
2023-08-17 16:49:16.583 | INFO     | pushcart.sources.rest_api.request:_build_no_auth_client:110 - Using no authentication.
2023-08-17 16:49:16.609 | DEBUG    | pushcart.sources.rest_api.request:_add_data_for_get_request:236 - Added JSON dump of data to GET request body.
2023-08-17 16:49:16.682 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:358 - Returning JSON response:
2023-08-17 16:49:16.682 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:360 - [{'page': 2, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 7, 'email': 'michael.lawson@reqres.in', 'first_name': 'Michael', 'last_name': 'Lawson', 'avatar': 'https://reqres.in/img/faces/7-image.jpg'}, {'id': 8, 'email': 'lindsay.fergu

In [6]:
split_tables["result"].show(truncate=False)

2023-08-17 16:49:17.965 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:202 - URL: https://reqres.in/api/users
2023-08-17 16:49:17.965 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:203 - Parameters: {'page': '2'}
2023-08-17 16:49:17.965 | INFO     | pushcart.sources.rest_api.request:_build_no_auth_client:110 - Using no authentication.
2023-08-17 16:49:18.030 | DEBUG    | pushcart.sources.rest_api.request:_add_data_for_get_request:236 - Added JSON dump of data to GET request body.
2023-08-17 16:49:18.114 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:358 - Returning JSON response:
2023-08-17 16:49:18.115 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:360 - [{'page': 2, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 7, 'email': 'michael.lawson@reqres.in', 'first_name': 'Michael', 'last_name': 'Lawson', 'avatar': 'https://reqres.in/img/faces/7-image.jpg'}, {'id': 8, 'email': 'lindsay.fergu

+----------------------------------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result_key                              |result_pos|result_col                                                                                                                                                                                                                                                                                                                                             

2023-08-17 16:49:19.388 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:202 - URL: https://reqres.in/api/users
2023-08-17 16:49:19.388 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_params:203 - Parameters: {'page': '2'}
2023-08-17 16:49:19.388 | INFO     | pushcart.sources.rest_api.request:_build_no_auth_client:110 - Using no authentication.
2023-08-17 16:49:19.412 | DEBUG    | pushcart.sources.rest_api.request:_add_data_for_get_request:236 - Added JSON dump of data to GET request body.
2023-08-17 16:49:19.473 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:358 - Returning JSON response:
2023-08-17 16:49:19.473 | DEBUG    | pushcart.sources.rest_api.request:_parse_api_response:360 - [{'page': 2, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 7, 'email': 'michael.lawson@reqres.in', 'first_name': 'Michael', 'last_name': 'Lawson', 'avatar': 'https://reqres.in/img/faces/7-image.jpg'}, {'id': 8, 'email': 'lindsay.fergu

In [7]:
split_tables.update(ps.split(split_tables["result"], ["result_col"]))
split_tables.update(ps.split(split_tables["result_col"], ["data"]))

split_tables["data"].show(truncate=False)


2023-08-17 16:49:19.552 | INFO     | pushcart.transformations.split_nested.spark:_split_root:296 - Creating dataframe to serve as base for splitting.
2023-08-17 16:49:19.671 | INFO     | pushcart.transformations.split_nested.spark:_create_split_table:254 - Splitting column result_col from original dataframe.
2023-08-17 16:49:19.797 | INFO     | pushcart.transformations.split_nested.spark:_split_children:337 - Created split dataframe for result_col column.
2023-08-17 16:49:19.813 | INFO     | pushcart.transformations.split_nested.spark:_split_root:296 - Creating dataframe to serve as base for splitting.
2023-08-17 16:49:19.864 | INFO     | pushcart.transformations.split_nested.spark:_create_split_table:254 - Splitting column data from original dataframe.
2023-08-17 16:49:19.920 | INFO     | pushcart.transformations.split_nested.spark:_split_children:337 - Created split dataframe for data column.
2023-08-17 16:49:20.660 | DEBUG    | pushcart.sources.rest_api.request:_update_url_and_param

+----------------------------------------+--------+-------------------------------------------------------------------------------------------+
|data_key                                |data_pos|data_col                                                                                   |
+----------------------------------------+--------+-------------------------------------------------------------------------------------------+
|28816157b5d87612c74d2a624d89841224209164|0       |{Lawson, https://reqres.in/img/faces/7-image.jpg, 7, Michael, michael.lawson@reqres.in}    |
|28816157b5d87612c74d2a624d89841224209164|1       |{Ferguson, https://reqres.in/img/faces/8-image.jpg, 8, Lindsay, lindsay.ferguson@reqres.in}|
|28816157b5d87612c74d2a624d89841224209164|2       |{Funke, https://reqres.in/img/faces/9-image.jpg, 9, Tobias, tobias.funke@reqres.in}        |
|28816157b5d87612c74d2a624d89841224209164|3       |{Fields, https://reqres.in/img/faces/10-image.jpg, 10, Byron, byron.fields@reqres.in}

23/08/17 16:49:27 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
